### 모델학습 준비

In [1]:
%pip install --upgrade pip
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.23.0%2Bcpu-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchaudio-2.8.0%2Bcpu-cp312-cp312-win_amd64.whl.metadata (7.4 kB)
Using cached https://download.pytorch.org/whl/cpu/torchvision-0.23.0%2Bcpu-cp312-cp312-win_amd64.whl (1.6 MB)
Using cached https://download.pytorch.org/whl/cpu/torchaudio-2.8.0%2Bcpu-cp312-cp312-win_amd64.whl (2.5 MB)

   ---------------------------------------- 0/2 [torchvision]
   ---------------------------------------- 0/2 [torchvision]
   ---------------------------------------- 0/2 [torchvision]
   ---------------------------------------- 0/2 [torchvision]
   ---------------------------------------- 0/2 [torchvision]
   ---------------------------------------- 0/2 [torchvision]
   ---------------------------------------- 0/

In [120]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [198]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")   #cuda장비인지 GPU장비인지 판단

In [124]:
BATCH_SIZE = 256
EPOCH = 30

In [126]:
transform_base = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()])

In [128]:
train_dataset = ImageFolder(root='./splitted/train',transform=transform_base)
val_dataset = ImageFolder(root='./splitted/val',transform=transform_base)

In [129]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True,
                                           num_workers=4)

In [130]:
val_loader = torch.utils.data.DataLoader(val_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True,
                                           num_workers=4)

### 모델 설계

In [135]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [136]:
class Net(nn.Module):
    def __init__(self):

        super(Net, self).__init__()


        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)

        self.fcl = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 33)

    def forward(self, x):

        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)

        x = x.view(-1,4096)
        x = self.fcl(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)

model_base = Net().to(DEVICE)
optimizer = optim.Adam(model_base.parameters(), lr=0.001)


#### 모델 학습을 위한 함수

In [144]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

#### 모델 평가를 위한 함수

In [148]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)


            test_loss += F.cross_entropy(output, 
                                        target, reduction='sum').item()

            pred = output.max(1, keepdim=True)[1]
            correct +=pred.eq(target.view_as(pred)).sum().item()


    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

        

### 모델 실행하기

In [154]:
import time
import copy

In [158]:
def train_baseline(model, train_loader, val_loader,
                  optimizer, num_epochs = 30):

    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(1, num_epochs + 1):
        since = time.time()
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, val_loader)

        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print('--------------epoch{}-----------'.format(epoch))

        print('train Loss : {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))

        print('val Loss : {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))

        print('completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    model.load_state_dict(best_model_wts)
    return model


base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)

torch.save(base,'baseline.pt')

--------------epoch1-----------
train Loss : 0.9949, Accuracy: 70.58%
val Loss : 1.0345, Accuracy: 69.27%
completed in 3m 46s
--------------epoch2-----------
train Loss : 0.7425, Accuracy: 76.85%
val Loss : 0.7880, Accuracy: 75.85%
completed in 3m 39s
--------------epoch3-----------
train Loss : 0.5632, Accuracy: 82.31%
val Loss : 0.6227, Accuracy: 80.57%
completed in 3m 40s
--------------epoch4-----------
train Loss : 0.4796, Accuracy: 85.54%
val Loss : 0.5432, Accuracy: 83.61%
completed in 3m 42s
--------------epoch5-----------
train Loss : 0.4043, Accuracy: 87.77%
val Loss : 0.4793, Accuracy: 85.27%
completed in 3m 57s
--------------epoch6-----------
train Loss : 0.3682, Accuracy: 88.45%
val Loss : 0.4527, Accuracy: 85.67%
completed in 3m 40s
--------------epoch7-----------
train Loss : 0.3301, Accuracy: 90.22%
val Loss : 0.4123, Accuracy: 87.08%
completed in 3m 53s
--------------epoch8-----------
train Loss : 0.2777, Accuracy: 91.55%
val Loss : 0.3688, Accuracy: 88.31%
completed in

### 모델평가를 위한 전처리

In [177]:
transform_base = transforms.Compose([transforms.Resize([64,64]),
                                    transforms.ToTensor()])

In [179]:
test_base = ImageFolder(root='./splitted/test',transform=transform_base)

In [181]:
test_loader = torch.utils.data.DataLoader(test_base,
                                              batch_size=BATCH_SIZE,
                                              shuffle=True,
                                              num_workers=4)

### 모델성능평가

In [187]:
from torch.serialization import add_safe_globals
add_safe_globals([Net])   # 신뢰하는 클래스로 Net 등록

baseline = torch.load('baseline.pt', map_location=DEVICE, weights_only=False)
baseline.eval()

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fcl): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=33, bias=True)
)

In [191]:
test_loss,test_accuracy = evaluate(baseline, test_loader)

In [192]:
print('baseline test acc: ',test_accuracy)

baseline test acc:  94.02929027412692
